In [1]:
# using CuArrays
using AutoGrad
using Deeplearning
using Statistics
using StatsBase
using Plots
import CSV

┌ Info: Recompiling stale cache file /Users/aliceylan/.julia/compiled/v1.1/Deeplearning/gtjFf.ji for Deeplearning [0d5bc7a0-39dd-11e9-2aab-59c8d4b467de]
└ @ Base loading.jl:1184
┌ Warning: Package Deeplearning does not have LinearAlgebra in its dependencies:
│ - If you have Deeplearning checked out for development and have
│   added LinearAlgebra as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Deeplearning
└ Loading LinearAlgebra into Deeplearning from project dependency, future warnings for Deeplearning are suppressed.


In [2]:
batch = 100
learning_rate = 10e-4

0.001

In [3]:
train = CSV.read("datasets/digit-recognizer/train.csv")
xtrain = []
ytrain = []

for i in 1:size(train)[1]
    max = maximum(train[i,2:end])+0.1
    normalized = reshape(train[i, 2:end]./(max), (28,28,1,1))
    push!(xtrain, normalized)
    push!(ytrain, train[i,1]+1)
end

println("Ready...")

Ready...


In [62]:
test = CSV.read("datasets/digit-recognizer/test.csv")
xtest = []
# ytest = []

for i in 1:size(test)[1]
    max = maximum(test[i,2:end])+0.1
    normalized = reshape(test[i, 1:end]./(max), (28,28,1,1))
    push!(xtest, normalized)
#     push!(ytest, test[i,1]+1)
end

println("Ready...")

Ready...


In [4]:
mnist_train = Deeplearning.minibatch(xtrain, ytrain, batch, shuffle=true)

Deeplearning.Data{Tuple{Array{Any,1},Array{Any,1}}}(Any[[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0] [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0] … [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0] [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]], Any[2 1 … 7 10], 100, 42000, false, 41901, 1:42000, true, (42000,), (42000,), Array{Any,1}, Array{Any,1})

In [63]:
mnist_test = Deeplearning.minibatch(xtest, ytest, batch, shuffle=true)

DimensionMismatch: DimensionMismatch("")

In [5]:
conv1 = Deeplearning.ConvolutionLayer((3,3,6), (2,2), (1,1), xavier, relu)((28,28,1,1))
conv2 = Deeplearning.ConvolutionLayer((3,3,16), (2,2), (1,1), xavier, relu)(conv1.telemetry.o)
dense1 = Deeplearning.FullyConnectedLayer(120, xavier, relu)(conv2.telemetry.o)
dense2 = Deeplearning.FullyConnectedLayer(84, xavier, relu)(dense1.telemetry.o)
dense3 = Deeplearning.FullyConnectedLayer(10, xavier, relu)(dense2.telemetry.o)
layers=[dense3,dense2,dense1,conv2,conv1]
println("Loaded")

Loaded


In [44]:
function f(x)
    x = cat(x...,dims=4)
    x = conv1(x,s=(2,2))    
    x = conv2(x,s=(2,2))    
    x = dense1(x)
    x = dense2(x)
    x = dense3(x)
    x = newsoftmax(x)
end

f (generic function with 1 method)

In [45]:
lenet5v2 = Deeplearning.Network(layers, f, loss=nll)

Deeplearning.Network(Any[FullyConnectedLayer(LayerTelemetry((10, 84), (10, 1, 1), (10, 1, 1)), P(Array{Float64,2}(10,84)), P(Array{Float64,2}(10,1)), relu), FullyConnectedLayer(LayerTelemetry((84, 120), (84, 1, 1), (84, 1, 1)), P(Array{Float64,2}(84,120)), P(Array{Float64,2}(84,1)), relu), FullyConnectedLayer(LayerTelemetry((120, 576), (120, 1, 1), (120, 1, 1)), P(Array{Float64,2}(120,576)), P(Array{Float64,2}(120,1)), relu), ConvolutionLayer(LayerTelemetry((3, 3, 6, 16), (1, 1, 16, 1), (6, 6, 16, 1)), P(Array{Float64,4}(3,3,6,16)), P(Array{Float64,4}(1,1,16,1)), relu), ConvolutionLayer(LayerTelemetry((3, 3, 1, 6), (1, 1, 6, 1), (13, 13, 6, 1)), P(Array{Float64,4}(3,3,1,6)), P(Array{Float64,4}(1,1,6,1)), relu)], f, Deeplearning.nll)

indices = sample(1:size(train)[1], size(train)[1], replace = false)
println("Ready...")

sample_batch_x, sample_batch_y = cat(xtrain[indices[(1):(100)]]..., dims=4), ytrain[indices[(1):(100)]]
println("Ready...")

using BenchmarkTools

dv = dff(sample_batch_x, sample_batch_y)

dvp = @parameters dv

In [ ]:
acc_train_plot = []
acc_test_plot = []
loss_train_plot = []
loss_test_plot = []

In [8]:
datastream = Deeplearning.progress(repeat(mnist_train, 1))

Deeplearning.Progress{Deeplearning.Repeat}(Deeplearning.Repeat(Deeplearning.Data{Tuple{Array{Any,1},Array{Any,1}}}(Any[[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0] [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0] … [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0] [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]], Any[2 1 … 7 10], 100, 42000, false, 41901, 1:42000, true, (42000,), (42000,), Array{Any,1}, Array{Any,1}), 1), 0, 0, 0x00000034fb6bc470, 0x0000000000000000, 0x000000003b9aca00, 80, 1.0, Inf)

In [10]:
di = collect(Iterators.take(mnist_train, 1))

1-element Array{Tuple{Array{Any,1},Array{Any,1}},1}:
 ([[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]  …  [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 

In [11]:
batch_x, batch_y = di[1]

(Any[[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]  …  [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0

In [14]:
newsoftmax(x) = cat([(exp10.((x[:,bi])))./(sum(exp10.((x[:,bi])))) for bi in 1:size(x)[end]]...,dims= 2)

newsoftmax (generic function with 1 method)

In [48]:
@btime lenet5v2(batch_x)

  28.446 ms (40109 allocations: 38.69 MiB)


10×100 Array{Float64,2}:
 0.0838721  0.0838721  0.0838725  …  0.0838722  0.0838721  0.0838722
 0.0838721  0.0838721  0.0838725     0.0838722  0.0838721  0.0838722
 0.164667   0.164667   0.164665      0.164667   0.164667   0.164667 
 0.106995   0.106995   0.106994      0.106995   0.106996   0.106995 
 0.112322   0.112323   0.112325      0.112323   0.112323   0.112323 
 0.112171   0.112171   0.11217    …  0.112171   0.11217    0.11217  
 0.0844836  0.0844836  0.0844825     0.0844838  0.0844836  0.0844836
 0.0838721  0.0838721  0.0838725     0.0838722  0.0838721  0.0838722
 0.0838721  0.0838721  0.0838725     0.0838722  0.0838721  0.0838722
 0.0838721  0.0838721  0.0838725     0.0838722  0.0838721  0.0838722

In [58]:
function acc(m::Deeplearning.Network, d::Deeplearning.Data;o...)
    total_cnt = 0
    total_smp = 0
    for (x_d, y_d) in d
        total_cnt+=Deeplearning.acc(m(x_d), y_d, average=false)
        total_smp+=length(y_d)
    end

    return total_cnt/total_smp
end

acc (generic function with 1 method)

In [60]:
@btime acc(lenet5v2, mnist_train)

  18.887 s (16862947 allocations: 15.87 GiB)


0.12857142857142856

In [25]:
using BenchmarkTools

In [26]:
@btime dv = @diff lenet5v2(cat(batch_x...,dims=4), batch_y)

  103.861 ms (140743 allocations: 99.99 MiB)


T(0.09852739331525731)

In [22]:
dparams = @parameters dv

10-element Array{Param,1}:
 P(Array{Float64,2}(10,1))    
 P(Array{Float64,2}(10,84))   
 P(Array{Float64,2}(84,1))    
 P(Array{Float64,2}(84,120))  
 P(Array{Float64,2}(120,1))   
 P(Array{Float64,2}(120,576)) 
 P(Array{Float64,4}(1,1,16,1))
 P(Array{Float64,4}(3,3,6,16))
 P(Array{Float64,4}(1,1,6,1)) 
 P(Array{Float64,4}(3,3,1,6)) 

In [23]:
dw = grad(dv, dparams[1])

10×1 Array{Float64,2}:
  0.0                  
  0.0                  
 -0.014414221874662863 
 -0.0026195316414336835
  0.005045085061008495 
  0.013196174025486744 
 -0.0039157665836509845
  0.0                  
  0.0                  
  0.0                  

In [ ]:
for (iteration, data) in enumerate(datastream)
    batch_x, batch_y = data

    dv = @diff lenet5v2(batch_x, batch_y)

    dparams = @parameters dv

    for pidx in 1:length(dparams[1:end-1])
        dw = grad(dv, dparams[pidx])
        dw .-=learning_rate*dw
    end
    
    if(iteration%sample_int==1)
        loss_train, loss_test = lenet5v2(mnist_train), lenet5v2(mnist_test)
        acc_train, acc_test = acc(lenet5v2, mnist_train), acc(lenet5v2, mnist_test)
    end
    
end

In [ ]:
plot([acc_plot], linewidth=2, title="Accuracy")

In [ ]:
plot([loss_plot], linewidth=2, title="Loss")